PyTorch

**Tensor**

원래는 1차원:벡터 2차원:행렬 3차원:텐서 이지만,
파이토치에서는 모든 데이터를 Tensor로 정의





실습1

In [6]:
import torch
import numpy as np

list_data = [[10,20],[30,40]]

tensor1 = torch.Tensor(list_data)

print(tensor1)
print(f'tensor type :{type(tensor1)}, tensor shape : {tensor1.shape}')
print(f'tensor dtype : {tensor1.dtype}, tensor device:{tensor1.device}')

#tensor.device : CPU GPU 등 텐서가 저장되는 위치

tensor([[10., 20.],
        [30., 40.]])
tensor type :<class 'torch.Tensor'>, tensor shape : torch.Size([2, 2])
tensor dtype : torch.float32, tensor device:cpu


In [10]:
if torch.cuda.is_available():
  tensor1 = tensor1.to("cuda")

print(f'tensor type :{type(tensor1)}, tensor shape : {tensor1.shape}')
print(f'tensor dtype : {tensor1.dtype}, tensor device:{tensor1.device}')

tensor type :<class 'torch.Tensor'>, tensor shape : torch.Size([2, 2])
tensor dtype : torch.float32, tensor device:cpu


In [ ]:
numpy_data = np.array([1,2])

tensor2 = torch.from_numpy(numpy_data).float() # numpy로 데이터 받으면 정수형이 default, float으로 casting해줘야함

tensor3 = torch.rand(2,2)
tensor4 = torch.randn(2,2) #randn 은 표준정규분포에서 값 추출 mean 0 , variance 1

numpy_from_tensor = tensor4.numpy() #tensor -> numpy 변환




**텐서의 연산**



In [12]:
tensor6 = torch.Tensor([[1,2,3],[4,5,6]])

tensor7 = torch.Tensor([[7,8,9],[10,11,12]])

#slicing은 기존 array및 numpy와 같음

**Element-wise product**

**mul**



In [13]:
tensor8 = tensor6.mul(tensor7) # Row & Column of Tensor should have same shape with each other.

**Matrix Multiplication**

**matmul**

In [14]:
ts9 = tensor6.matmul(tensor7.view(3,2)) # view  = reshape

**Tensor Concatenate**


In [18]:

ts_cat = torch.cat([tensor6,tensor7], dim=0) # 첫번째 차원을 기준으로 합치기 행추가
ts_cat2 = torch.cat([tensor6,tensor7],dim=1) # 두번째 차원기준으로 합치기 (열추가)

**Tensor Data processing**

In [17]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

x_train = torch.Tensor([1,2,3,4,5,6]).view(6,1)
y_train = torch.Tensor([3,4,5,6,7,8]).view(6,1)

dataset = TensorDataset(x_train,y_train) #DataSet만들기
dataloader = DataLoader(dataset,batch_size=32, shuffle = True) #DataSet에서 DataLoading process를 담당하는 DataLoader


# **Torch.nn**
**torch.nn 패키지 : neural network 패키지**

**Neural Network Learning Exercise**


In [21]:
import numpy as np

loaded_data = np.loadtxt('/content/drive/MyDrive/LEC06_TrainData.csv',delimiter=',')

x_train_np = loaded_data[:,0:-1]
y_train_np = loaded_data[:,[-1]]




In [27]:
import torch
from torch import nn

x_train = torch.Tensor(x_train_np)
y_train = torch.Tensor(y_train_np)

**학습모델 구성**

In [23]:
class MyLinearRegressionModel(nn.Module):
  def __init__(self,input_nodes):
    super().__init__()
    self.linear_stack = nn.Sequential(
        nn.Linear(input_nodes,1)
    )
    #Sequential이 박스
    #nn.Linear가 한개 레이어, 선형이아닌 다른 레이어도 추가 가능

  def forward(self,data):
    pred = self.linear_stack(data)
    return pred

In [26]:
model = MyLinearRegressionModel(3)

for name, child in model.named_children():
  for param in child.parameters():
    print(name,"##",param)

linear_stack ## Parameter containing:
tensor([[ 0.2360, -0.4935,  0.1728]], requires_grad=True)
linear_stack ## Parameter containing:
tensor([-0.5087], requires_grad=True)


**손실함수 및 옵티마이저 설정**

In [31]:
loss_function = nn.MSELoss()

optimizer = torch.optim.SGD(model.parameters(),lr=1e-2)

lost_list = []

nums_epoch=2000

for epoch in range(nums_epoch):
  pred = model(x_train)
  loss = loss_function(pred,y_train)

  lost_list.append(loss.item())
  optimizer.zero_grad() #gard 초기화, 아마 optimizer만들때 model.parameters()넘겨준거 정보 바탕으로 grad 구성하는듯.
  loss.backward() ##여기서 loss가 optimizer한테 grad 계산해서 넘겨주는 과정을 모르겠음. 서로 연결된게 없는데?
  optimizer.step()

  if(epoch%100==0):
    print(int(100*(epoch+1)/nums_epoch),"% --> current loss :", loss.item())




0 % --> current loss : 2.0842586337388447e-11
5 % --> current loss : 2.0842586337388447e-11
10 % --> current loss : 2.0979011930433167e-11
15 % --> current loss : 2.0842586337388447e-11
20 % --> current loss : 2.0842586337388447e-11
25 % --> current loss : 2.0979011930433167e-11
30 % --> current loss : 2.0842586337388447e-11
35 % --> current loss : 2.0842586337388447e-11
40 % --> current loss : 2.0979011930433167e-11
45 % --> current loss : 2.0842586337388447e-11
50 % --> current loss : 2.0842586337388447e-11
55 % --> current loss : 2.0979011930433167e-11
60 % --> current loss : 2.0842586337388447e-11
65 % --> current loss : 2.0842586337388447e-11
70 % --> current loss : 2.0979011930433167e-11
75 % --> current loss : 2.0842586337388447e-11
80 % --> current loss : 2.0842586337388447e-11
85 % --> current loss : 2.0979011930433167e-11
90 % --> current loss : 2.0842586337388447e-11
95 % --> current loss : 2.0842586337388447e-11


In [36]:
x_test = torch.Tensor([ [5, 5, 0], [2, 3, 1], [-1, 0, -1], [10, 5, 2], [4, -1, -2] ])

label = [ 2*data[0] -3*data[1] + 2*data[2]  for data in x_test ]

pred = model(x_test)
# pred = model.forward(x_test) 가 아닌이유?
#아마 Module클래스가 그렇게 구성되었을듯. Model(data)하면 forward된 값이 나오도록?

print(pred.view(5,))
print("-----------------")
print(label)

tensor([-5.0000, -3.0000, -4.0000,  9.0000,  7.0000], grad_fn=<ViewBackward0>)
-----------------
[tensor(-5.), tensor(-3.), tensor(-4.), tensor(9.), tensor(7.)]
